# Wstęp do studium przypadku

## Web API

In [ ]:
import json

In [ ]:
# Słownik Pythona...
user_dict = {"imię i nazwisko": "Anna Kowalska",
             "wiek": 23,
             "zamężna": False,
             "dzieci": None,
             "zainteresowania": ["góry", "książki"]}

In [ ]:
# ...przekonwertowany na łańcuch w formacie JSON
# za pomocą json.dumps. Parametr "indent" jest
# opcjonalny i służy upiększeniu wydruku.
user_json = json.dumps(user_dict, indent=4)
print(user_json)

In [ ]:
# Konwersja łańcucha JSON z powrotem do rodzimej struktury danych Pythona
json.loads(user_json)

In [ ]:
import requests

In [ ]:
response = requests.get("https://pypi.org/pypi/pandas/json")
response.status_code

In [ ]:
# response.json()

In [ ]:
releases = []
for version, files in response.json()['releases'].items():
    releases.append(f"{version}: {files[0]['upload_time']}")
releases[:3]  # wyświetlenie pierwszych 3 elementów listy

## Bazy danych

In [ ]:
import urllib.parse

In [ ]:
urllib.parse.quote_plus("hasło")

In [ ]:
# Zacznijmy od importowania
import sqlite3
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
# Nasze zapytanie SQL: "wybierz wszystkie kolumny z tabeli packages"
sql = "SELECT * FROM packages"

In [ ]:
# Opcja 1: sterownik bazy danych (sqlite3 jest częścią biblioteki standardowej)
# Użycie połączenia jako menedżera kontekstu powoduje, że transakcja jest
# automatycznie zatwierdzana lub wycofywana w przypadku wystąpienia błędu.
with sqlite3.connect("packagetracker/packagetracker.db") as con:
    cursor = con.cursor()  # Do wykonywania zapytań SQL potrzebny jest kursor
    result = cursor.execute(sql).fetchall()  # Zwrócenie wszystkich rekordów
result

In [ ]:
# Opcja 2: SQLAlchemy
# "create_engine" oczekuje łańcucha połączenia z bazą danych.
# Możemy tu wykonać zapytanie jako metodę obiektu połączenia.
engine = create_engine("sqlite:///packagetracker/packagetracker.db")
with engine.connect() as con:
    result = con.execute(sql).fetchall()
result

In [ ]:
# Opcja 3: pandas
# Podanie nazwy tabeli do "read_sql" powoduje odczytanie całej tabeli.
# pandas wymaga silnika SQLAlchemy, którego używamy ponownie
# z poprzedniego przykładu.
df = pd.read_sql("packages", engine, index_col="package_id")
df

In [ ]:
# "read_sql" akceptuje również zapytanie SQL
pd.read_sql(sql, engine, index_col="package_id")

In [ ]:
# # Metoda DataFrame "to_sql" zapisuje obiekty DataFrame do tabel.
# "if_exists" musi mieć wartość "fail", "append" lub "replace"
# i określa, co się stanie, jeśli tabela już istnieje.
df.to_sql("packages2", con=engine, if_exists="append")

In [ ]:
# Poprzednie polecenie utworzyło nową tabelę "packages2"
# i wstawiło do niej rekordy z DataFrame df, co możemy
# sprawdzić, odczytując jej zawartość.
pd.read_sql("packages2", engine, index_col="package_id")

In [ ]:
# Pozbądźmy się tabeli, uruchamiając polecenie
# "drop table" poprzez SQLAlchemy
with engine.connect() as con:
    con.execute("DROP TABLE packages2")

In [ ]:
# Zacznijmy od zaimportowania funkcji text SQLAlchemy
from sqlalchemy.sql import text

In [ ]:
# ":package_id" jest zamiennikiem
sql = """
SELECT v.uploaded_at, v.version_string
FROM packages p
INNER JOIN package_versions v ON p.package_id = v.package_id
WHERE p.package_id = :package_id
ORDER BY v.uploaded_at
"""

In [ ]:
# Poprzez SQLAlchemy
with engine.connect() as con:
    result = con.execute(text(sql), package_id=1).fetchall()
result[:3]  # Wypisanie pierwszych 3 rekordów

In [ ]:
# Poprzez pandas
pd.read_sql(text(sql), engine, parse_dates=["uploaded_at"],
            params={"package_id": 1},
            index_col=["uploaded_at"]).head(3)

## Wyjątki

In [ ]:
def print_reciprocal(number):
    result = 1 / number
    print(f"Liczba odwrotna to: {result}")

In [ ]:
print_reciprocal(0)  # Spowoduje to wystąpienie błędu

In [ ]:
def print_reciprocal(number):
    try:
        result = 1 / number
    except Exception as e:
        # "as e" udostępnia obiekt Exception jako zmienną "e"
        # "repr" oznacza "drukowalną reprezentację" obiektu
        # i zwraca łańcuch znaków z komunikatem o błędzie
        print(f"Wystąpił błąd: {repr(e)}")
        result = "nd."
    else:
        print("Nie było błędu!")
    finally:
        print(f"Liczba odwrotna to: {result}")

In [ ]:
print_reciprocal(10)

In [ ]:
print_reciprocal("a")

In [ ]:
print_reciprocal(0)

In [ ]:
def print_reciprocal(number):
    try:
        result = 1 / number
        print(f"Liczba odwrotna to: {result}")
    except (TypeError, ZeroDivisionError):
        print("Wpisz dowolną liczbę z wyjątkiem 0.")

In [ ]:
print_reciprocal("a")

In [ ]:
def print_reciprocal(number):
    try:
        result = 1 / number
        print(f"Liczba odwrotna to: {result}")
    except TypeError:
        print("Wpisz liczbę.")
    except ZeroDivisionError:
        print("Zero nie ma zdefiniowanej liczby odwrotnej.")

In [ ]:
print_reciprocal("a")

In [ ]:
print_reciprocal(0)